<a href="https://colab.research.google.com/github/reic/colab_python/blob/main/crawer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title 載入基礎函數
#@markdown 開啟暫存標目標

import requests
import os,re
from bs4 import BeautifulSoup

try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")



In [ ]:
#@title 品書閣 專用
#@markdown 查看目錄=>章節列表
url="https://tw.pinsuge.com/index/22.html" #@param {type:'string'}
sites=url[:url.find("/",8)]
req = requests.get(url)
#@markdown 在 stop_word 之後為相關的廣告內容
stop_word="請大家收藏" #@param {type:'string'}
#@markdown 若章節分成兩個頁面的處理字元
next_word="點下一章繼續" #@param {type:'string'}
soup=BeautifulSoup(req.text)

soup
articles=soup.find_all(name="a")
output_name=soup.find("h1").getText()
links=[]

for itm in articles[1:]:
  if not itm.get("title"):
    continue 
  links.append([itm.get("title",),"{}{}".format(sites,itm.get("href"))])


def get_content(content):
  to_text=''
  for itm in content[:-1]:
    txt=itm.getText()
    if "一秒記住" in txt:
      continue
    if stop_word in txt:
      break    
    to_text+="%s \n" %txt
  return to_text

def get_soup(url):
  return BeautifulSoup(requests.get(url).text)

def cont_chapter_check(soup,text):
  return (len(soup.find_all(name="p",string=re.compile(text))) >0)

def cont_chapter_link(soup):
  return soup.find("a",text="下一章").get("href")

index=1
for link in links:  
  soup=get_soup(link[1])
  to_text=''
  to_text+="%s \n" %link[0]
  print(link[0])
  to_text+=get_content(soup.find(name="div",id="content").find_all(name="p"))
  
  if cont_chapter_check(soup,next_word):
    reg2=requests.get("{}{}".format(sites,cont_chapter_link(soup)))
    to_text+=get_content(BeautifulSoup(reg2.text).find(name="div",id="content").find_all(name="p"))
  # print(to_text)

  with open("%03d.txt" % index,mode="w",encoding="utf-8") as f:
    f.write(to_text)
  index+=1

files=os.listdir()
files=[file for file in files if file.endswith(".txt")]
files.sort()
files
for file in files:
  os.system("cat {}>> ../{}.txt".format(file,output_name))

from google.colab import files
files.download('../{}.txt'.format(output_name)) 
os.system("rm -fr *")